In [1]:
import pandas as pd
import numpy as np
import bs4
from bs4 import BeautifulSoup
import requests
from urllib.request import Request, urlopen
from bs4 import BeautifulSoup

In [2]:
# objective: classify cannabis' dominant strain [Indica, Sativa] 
# from chemical data

In [3]:
# Gather data ---------------------------------------------------

In [4]:
# CSV Chemical Data
loc = '~/Desktop/portfolio/CannAnxi/Canna.csv'
df = pd.read_csv(loc)

In [5]:
# cleaning CSV data -- dropping columns not of interest
to_drop = [
    'OCPID',
    'Δ8-THC',
    'Strain',
    'SampleID',
    'DateRecorded',
    'ReportType',
    'ChemicalLab',
    'H2O',
    'TotalTHC',
    'GeneticLab',
    'Sample',
    'SampleURL',
    'Organism',
    'OrganismURL',
    'Project',
    'ProjectURL',
    'Study',
    'StudyURL',
    'Run',
    'RunURL',
    'DatePublished',
    'Spots',
    'Bases',
    'Size',
    'Notes',
]

# save strain names 
namesDf = df['Strain']
namesDf = namesDf[:255]
# display(namesDf)

# drop features
df = df.drop(to_drop, axis=1)

In [6]:
# cleaning data -- cut rows --> only include chemical data (rows == 255)
df = df[:255]

In [7]:
# cleaning data -- replace NaN vals with 0 
# i.e. chemical is not present in sample
df = df.fillna(0)

In [8]:
# class labels
# for name in namesDf:
#     print(name)

In [51]:
# leafly --> HTTPError: HTTP Error 403: Forbidden
# Cannasos success parsing -- no success extracting
# 


url = 'https://potguide.com/strain-profiles/blue-dream/'

parse_html_url = urlopen(url)
html_page = parse_html_url.read()
parse_html_url.close()

strain = BeautifulSoup(html_page, "html.parser")

vals = strain.findAll("div", {"class":"strain-profile-basic-content", "itemprop":"text"})
#val = val.get_text()
for val in vals:
    val = val.get_text()
    if val.find("indica") != -1:
        print("indica")
        break
    elif val.find("sativa") != -1:
        print("sativa")
        break
    else:
        pass
    
    

sativa


In [9]:
# Web Scrape class
for name in namesDf:
    url = 'https://www.leafly.com/strains/'
    url = url + str(name).replace(' ', '-').lower()
    print(url)
    try:
        parse_html_url = urlopen(url)
        print('success')
        html_page = parse_html_url.read()
        parse_html_url.close()
        
        strain = BeautifulSoup(html_page, "html.parser")
        
        #val = strain.findAll('h2')
        print('success')
        
    except:
        print(name, 'sucks')
        pass


https://www.leafly.com/strains/chem-4
CHEM 4 sucks
https://www.leafly.com/strains/z7
Z7 sucks
https://www.leafly.com/strains/acdc/harlequin
ACDC/Harlequin sucks
https://www.leafly.com/strains/harleywreck
HarleyWreck sucks
https://www.leafly.com/strains/sour-amnesia
Sour Amnesia sucks
https://www.leafly.com/strains/jack-herer
Jack Herer sucks
https://www.leafly.com/strains/critical-cheese
Critical Cheese sucks
https://www.leafly.com/strains/sour-tangie
Sour Tangie sucks
https://www.leafly.com/strains/wildfire
WildFire sucks
https://www.leafly.com/strains/papaya
Papaya sucks
https://www.leafly.com/strains/cbd-critical-mass
CBD Critical Mass sucks
https://www.leafly.com/strains/moon-puppies
Moon Puppies sucks
https://www.leafly.com/strains/master-kush
Master Kush sucks
https://www.leafly.com/strains/romulan-grapefruit
Romulan Grapefruit sucks
https://www.leafly.com/strains/golden-pineapple
Golden Pineapple sucks
https://www.leafly.com/strains/kush-med
Kush Med sucks
https://www.leafly.com

Mountain Girl Lemon sucks
https://www.leafly.com/strains/jager
Jager sucks
https://www.leafly.com/strains/marionberry-kush
Marionberry Kush sucks
https://www.leafly.com/strains/cookies-and-cream
Cookies And Cream sucks
https://www.leafly.com/strains/sweeties
Sweeties sucks
https://www.leafly.com/strains/phyre-glue
Phyre Glue sucks
https://www.leafly.com/strains/pkt
Pkt sucks
https://www.leafly.com/strains/mountain-girl
Mountain Girl sucks
https://www.leafly.com/strains/harley-wreck-5
Harley Wreck 5 sucks
https://www.leafly.com/strains/tahoe-dream
Tahoe Dream sucks
https://www.leafly.com/strains/sugarloaf
Sugarloaf sucks
https://www.leafly.com/strains/telenovela
Telenovela sucks
https://www.leafly.com/strains/cbd-green-crack
Cbd Green Crack sucks
https://www.leafly.com/strains/berry-white
Berry White sucks
https://www.leafly.com/strains/blackberry-cobbler
Blackberry Cobbler sucks
https://www.leafly.com/strains/city-of-god
City Of God sucks
https://www.leafly.com/strains/ancient-art
Anci